In [4]:
import math
import numpy as np
import re
import argparse
import os
import glob
from pathlib import Path
import time

In [8]:
class Indice_Invertido:
    
    #
    def formataString(s):
        s = s.lower()
        s = re.sub("[:,'|.@()?!#$&]"," ", s)
        s = s.replace("\n", " ")
        s = re.sub('[^A-Za-z0-9 ]+', '', s)
        strings = s.split()
        return strings

    
    def indiceInvertido(arquivos):
        vocabulario = []

        for k, v in arquivos.items():
            ignorarIndice = []
            for i in range(len(v)):
                frequenciaNesteDoc = 1
                if (ignorarIndice.count(i) != 0):
                    continue
                    
                for j in range(i+1, len(v)):
                    if (v[i] == v[j]):
                        frequenciaNesteDoc+=1
                        ignorarIndice.append(j)
                        
                vocabulario.append([v[i], k, frequenciaNesteDoc])
        indInv = {}

        for val in vocabulario:
            if val[0] in indInv.keys():
                dados = indInv[val[0]]
                dados.append([val[1], val[2]])
                indInv[val[0]] = dados
            else:
                dados = [[val[1],val[2]]]
                indInv[val[0]] = dados
                
        return indInv

    
    def coordenadas(indInv, numDocs):
        coords = np.zeros((len(indInv), numDocs))
        indicePalavra = 0;

        for k, v in indInv.items():
            nx = len(v)
            idf = math.log(numDocs/nx)
            
            for dados in v:
                w = dados[1] * idf
                coords[indicePalavra][dados[0]] = w

            indicePalavra+=1

        return coords

    #essa função deu erro em relação ao k
    def formataQuery(indInv, indQ, numDocs):
        Q = np.zeros((len(indInv),1))
        indicePalavra = 0;
    
        for k, v in indInv.items():
            if k in indQ.keys():
                nx = len(v)
                idf = math.log(numDocs/nx)
                w = (indQ[k])[0][1] * idf
                Q[indicePalavra] = w
            indicePalavra+=1
        return Q

    
    def verificaConsulta(coords, Q):
        resultado = {}
        Q = Q.reshape((len(Q),1))

        for i in range(np.shape(coords)[1]):
            vec = coords[:, i]
            vec = vec.reshape((len(vec),1))
            similaridade = (vec.T @ Q)/(np.linalg.norm(vec) * np.linalg.norm(Q))
            resultado[i] = similaridade[0][0]
        return resultado